In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Activation, BatchNormalization, LeakyReLU, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

def discriminator_model(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    x = Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(filters=128, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(filters=256, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(filters=512, kernel_size=(4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Flatten and output layer
    x = Flatten()(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs, name='Discriminator')
    return model

# Assuming load_images and generator_model functions are already defined

# Define generator_containing_discriminator_multiple_outputs
def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=(256, 256, 3))
    generated_images = generator(inputs)
    discriminator.trainable = False
    outputs = discriminator(generated_images)
    return Model(inputs, [generated_images, outputs])

#change generator model variable as required
g = generator_model()
d = discriminator_model()
d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

d.trainable = True
d.compile(optimizer=d_opt, loss=wasserstein_loss)
d.trainable = False
loss = [perceptual_loss, wasserstein_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True


epoch_num = 100
batch_size = 16
critic_updates = 5

for epoch in range(epoch_num):
    print('epoch: {}/{}'.format(epoch, epoch_num))
    print('batches: {}'.format(x_train.shape[0] / batch_size))

    permutated_indexes = np.random.permutation(x_train.shape[0])

    for index in range(int(x_train.shape[0] / batch_size)):
        batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
        image_blur_batch = x_train[batch_indexes]
        image_full_batch = y_train[batch_indexes]

        generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

        for _ in range(critic_updates):
            d_loss_real = d.train_on_batch(image_full_batch, np.ones((batch_size, 1)))
            d_loss_fake = d.train_on_batch(generated_images, -np.ones((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

        d.trainable = False
        d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, np.ones((batch_size, 1))])
        d.trainable = True


NameError: name 'generator_model' is not defined